In [1]:
import numpy as np
import numpy.linalg as la
from scipy.linalg import logm

In [2]:
X  = .5*np.array([[0, 1 ], [ 1 , 0]])
Y  = .5*np.array([[0,-1j], [ 1j, 0]])
Z  = .5*np.array([[1, 0 ], [ 0 ,-1]])
YL = .5*np.array([[0, 1 ], [-1 , 0]])
YR = .5*np.array([[0,-1 ], [ 1 , 0]])
SS = np.kron(X,X) + np.kron(YL,YR) + np.kron(Z,Z)
I  = np.eye(2)

In [3]:
def pauli_string(pauli_list, idx_list, L):
    p_list = [I for x in range(L)]
    for p, idx in zip(pauli_list, idx_list):
        p_list[idx] = p
    mat = 1
    for p in p_list:
        mat = np.kron(mat, p)
    return mat
def Heisenberg(W_list):
    H = 0
    L = len(W_list)
    for i, W in enumerate(W_list):
        if i < L-1:
            H += pauli_string([SS],   [i], L-1)
        H += pauli_string([W*Z], [i], L)
    return H

def get_Sa(a, L):
    Sa = 0
    for i in range(a):
        Sa += pauli_string([Z], [i], L)
    return Sa

In [34]:
def bin_split(n, L, idx_list):
    a, b = '', ''
    s = bin(n)[2:].zfill(L)
    for i in range(L):
        if i in idx_list:
            b += s[i]
        else:
            a += s[i]
    return int(a,2), int(b,2)

def partial_trace(O, idx_list):
    L,_= O.shape
    N  = int(np.log2(L))
    Nb = len(idx_list)
    Na = N - Nb
    L_b = 2**Nb
    L_a = 2**(N - Nb)
    
    O_a = np.zeros((L_a,L_a))
    for i in range(L):
        for j in range(L):
            i_a, i_b = bin_split(i, N, idx_list)
            j_a, j_b = bin_split(j, N, idx_list)
            if i_b == j_b:
                O_a[i_a,j_a] += O[i,j]
    return O_a

def find_eig_idx(E_list, E):
    return min(range(len(E_list)), key=lambda i: abs(E_list[i]-E))

def entanglement_entropy(psi, a):
    L,_  = psi.shape
    N    = int(np.log2(L))
    rho  = psi@psi.transpose()
    idx_list = list(range(a,N)) if a < N//2 else list(range(a))
    rhoA = partial_trace(rho, idx_list)
    return -np.trace(rhoA@logm(rhoA)).real

def bip_spin_fluc(psi):
    L,_ = psi.shape
    N = int(np.log2(L))
    Sa = get_Sa(N//2, N)
    v = psi.transpose()@Sa@Sa@psi
    e = psi.transpose()@Sa@psi
    return (v - e*e)[0,0]

def participation_entropy(psi):
    L, _ = psi.shape
    PE = 0
    for i in range(L):
        p = psi[i,0]**2
        if p > 0:
            PE -= p*np.log(p)
    return PE

def pretty_print(U, precision=3):
    with np.printoptions(precision=precision, suppress=True):
        print(U)

In [40]:
L = 8
W_list = [-4.42409, 2.93631, -2.53264, 0.745125, 6.9311, -1.17513, -1.9877, -2.11398]
#W_list = np.zeros(L)
H = Heisenberg(W_list)
E_list, U_list = la.eigh(H)

In [43]:
E = 4.89472

i = find_eig_idx(E_list, E)
psi = U_list[:,i].reshape((2**L,1))
print("ep = ", (E_list[i] - min(E_list))/(max(E_list)-min(E_list)))
print("E = ", E_list[i])
print("EEs:")
for a in range(1,L):
    print(entanglement_entropy(psi, a))
print("bip spin = ", bip_spin_fluc(psi))
print("PE       = ", participation_entropy(psi))
#print("psi:")
#pretty_print(psi, 5)

ep =  0.7213662197834295
E =  4.8956904733602835
EEs:
0.027653256121587116
0.0017547901583794862
logm result may be inaccurate, approximate err = 3.448777614999912e-13
2.0362952220786415e-05
5.321509709561186e-08
1.1770014118302592e-09
4.1521791510283496e-11
2.010052753228452e-12
bip spin =  2.5605029208009e-09
PE       =  0.028416612573925977


In [32]:
E = psi.transpose()@H@psi
var = psi.transpose()@H@H@psi
print(var - E*E)

[[2.66453526e-15]]
